# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [3]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')

In [7]:
filepath = song_files[0]

In [8]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARGSJW91187B9B1D6B,35.21962,North Carolina,-80.01955,JennyAnyKind,218.77506,1,SOQHXMF12AB0182363,Young Boy Blues,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = df[['song_id', 'title', 'artist_id','year', 'duration']].values
song_data[0]

array(['SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0,
       218.77506], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
song_table_insert

'\nINSERT INTO songs (song_id, title, artist_id, year, duration)                  VALUES (%s, %s, %s, %s, %s)\n'

In [11]:
cur.execute(song_table_insert, song_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = df[['artist_id', 'artist_name', 'artist_location','artist_latitude', 'artist_longitude']].values
artist_data[0]

array(['ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962,
       -80.01955], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files('data/log_data')

In [15]:
filepath = log_files[0]

In [16]:
df =  pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,689,None,200,1542592468796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
1,Explosions In The Sky,Logged In,Adelyn,F,0,Jordan,497.47546,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.540131e+12,458,Your Hand In Mine,200,1542592496796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",7
2,None,Logged In,Adelyn,F,1,Jordan,NaN,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,Logout,1.540131e+12,458,None,307,1542592497796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",7
3,None,Logged Out,None,None,2,None,NaN,free,None,GET,Home,NaN,458,None,200,1542592500796,None,
4,None,Logged Out,None,None,3,None,NaN,free,None,PUT,Login,NaN,458,None,307,1542592501796,None,


In [17]:
df=df[df.page == 'NextSong']

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df = df["ts"]
df.head()

1    1542592496796
6    1542592893796
7    1542593318796
8    1542593641796
9    1542593868796
Name: ts, dtype: int64

In [19]:
t = pd.to_datetime(df)
t.head()

1   1970-01-01 00:25:42.592496796
6   1970-01-01 00:25:42.592893796
7   1970-01-01 00:25:42.593318796
8   1970-01-01 00:25:42.593641796
9   1970-01-01 00:25:42.593868796
Name: ts, dtype: datetime64[ns]

In [18]:
time_data = [df, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ["timestamp","hour", "day", "weekofyear", "month", "year", "weekday"]

In [19]:
dft=pd.DataFrame(time_data).T

In [20]:
dft.columns = column_labels

In [21]:
time_df = dft
time_df.head()

,timestamp,hour,day,weekofyear,month,year,weekday
1,1542592496796,0,1,1,1,1970,3
6,1542592893796,0,1,1,1,1970,3
7,1542593318796,0,1,1,1,1970,3
8,1542593641796,0,1,1,1,1970,3
9,1542593868796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
df =  pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,689,None,200,1542592468796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
1,Explosions In The Sky,Logged In,Adelyn,F,0,Jordan,497.47546,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.540131e+12,458,Your Hand In Mine,200,1542592496796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",7
2,None,Logged In,Adelyn,F,1,Jordan,NaN,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,Logout,1.540131e+12,458,None,307,1542592497796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",7
3,None,Logged Out,None,None,2,None,NaN,free,None,GET,Home,NaN,458,None,200,1542592500796,None,
4,None,Logged Out,None,None,3,None,NaN,free,None,PUT,Login,NaN,458,None,307,1542592501796,None,


In [23]:
df.iloc[3].name

3

In [24]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df.head()

,userId,firstName,lastName,gender,level
0,32,Lily,Burns,F,free
1,7,Adelyn,Jordan,F,free
2,7,Adelyn,Jordan,F,free
3,,None,None,None,free
4,,None,None,None,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [46]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
#     print(songid)
#     print(artistid)
    # insert songplay record
    songplay_data = (row.ts, row.userId,row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [27]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.